In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import pandas as pd
import sklearn
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)

for module in mpl,np,pd,sklearn,tf,keras:
    print(module.__name__,module.__version__)

2.1.0
sys.version_info(major=3, minor=7, micro=4, releaselevel='final', serial=0)
matplotlib 3.1.1
numpy 1.17.2
pandas 0.25.1
sklearn 0.21.3
tensorflow 2.1.0
tensorflow_core.python.keras.api._v2.keras 2.2.4-tf


In [2]:
from sklearn.datasets import fetch_california_housing

housing= fetch_california_housing()

In [3]:
from sklearn.model_selection import train_test_split

x_train_all,x_test,y_train_all,y_test = train_test_split(
    housing.data, housing.target, random_state =7)

x_train,x_valid,y_train,y_valid = train_test_split(
    x_train_all,y_train_all,random_state=11)

print(x_train_all.shape,y_train_all.shape)
print(x_train.shape,y_train.shape)
print(x_valid.shape,y_valid.shape)
print(x_test.shape,y_test.shape)

(15480, 8) (15480,)
(11610, 8) (11610,)
(3870, 8) (3870,)
(5160, 8) (5160,)


In [4]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_valid_scaled = scaler.fit_transform(x_valid)
x_test_scaled = scaler.fit_transform(x_test)

In [5]:
output_dir = "generate_csv"

if not os.path.exists(output_dir):
    os.mkdir(output_dir)
    
def save_to_csv(output_dir, data, name_prefix,
                header=None, n_parts=10):
    
    path_format = os.path.join(output_dir, "{}_{:02d}.csv")
    filenames = []
    
    for file_idx, row_indices in enumerate(
        np.array_split(np.arange(len(data)),n_parts)):
        part_csv = path_format.format(name_prefix, file_idx)
        filenames.append(part_csv)
        
        with open(part_csv, "wt", encoding="utf-8") as f:
            if header is not None:
                f.write(header + "\n")
            
            for row_index in row_indices:
                f.write(",".join(
                    [repr(col) for col in data[row_index]]))

                f.write('\n')
    
    return filenames

train_data = np.c_[x_train_scaled, y_train]
valid_data = np.c_[x_valid_scaled, y_valid]
test_data = np.c_[x_test_scaled, y_test]

header_cols = housing.feature_names + ["MidianHouseVaiue"]

header_str = ",".join(header_cols)

In [6]:
print(housing.feature_names)

['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']


In [7]:
train_filenames = save_to_csv(output_dir,train_data,'train',
                              header_str,n_parts=20)

valid_filenames = save_to_csv(output_dir,valid_data,'valid',
                              header_str,n_parts=20)

test_filenames = save_to_csv(output_dir,test_data,'test',
                              header_str,n_parts=20)

In [8]:
import pprint
print("train_filenames")
pprint.pprint(train_filenames)
print("valid_filenames")
pprint.pprint(valid_filenames)
print("test_filenames")
pprint.pprint(test_filenames)

train_filenames
['generate_csv/train_00.csv',
 'generate_csv/train_01.csv',
 'generate_csv/train_02.csv',
 'generate_csv/train_03.csv',
 'generate_csv/train_04.csv',
 'generate_csv/train_05.csv',
 'generate_csv/train_06.csv',
 'generate_csv/train_07.csv',
 'generate_csv/train_08.csv',
 'generate_csv/train_09.csv',
 'generate_csv/train_10.csv',
 'generate_csv/train_11.csv',
 'generate_csv/train_12.csv',
 'generate_csv/train_13.csv',
 'generate_csv/train_14.csv',
 'generate_csv/train_15.csv',
 'generate_csv/train_16.csv',
 'generate_csv/train_17.csv',
 'generate_csv/train_18.csv',
 'generate_csv/train_19.csv']
valid_filenames
['generate_csv/valid_00.csv',
 'generate_csv/valid_01.csv',
 'generate_csv/valid_02.csv',
 'generate_csv/valid_03.csv',
 'generate_csv/valid_04.csv',
 'generate_csv/valid_05.csv',
 'generate_csv/valid_06.csv',
 'generate_csv/valid_07.csv',
 'generate_csv/valid_08.csv',
 'generate_csv/valid_09.csv',
 'generate_csv/valid_10.csv',
 'generate_csv/valid_11.csv',
 'genera

In [9]:
# 1. filename -> dataset
# 2. read file -> dataset -> datasets -> merge
# 3. parse csv

filename_dataset = tf.data.Dataset.list_files(train_filenames)

for filename in filename_dataset:
    print(filename)

tf.Tensor(b'generate_csv/train_09.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_13.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_10.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_17.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_16.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_00.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_11.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_06.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_02.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_15.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_03.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_19.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_07.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_05.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_04.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_12.csv', 

In [10]:
n_readers = 5

dataset = filename_dataset.interleave(
    lambda filename: tf.data.TextLineDataset(filename).skip(1),
    cycle_length = n_readers )

for line in dataset.take(15):
    print(line.numpy())

b'0.6363646332204844,-1.0895425985107923,0.09260902815633619,-0.20538124656801682,1.2025670451003232,-0.03630122549633783,-0.6784101660505877,0.182235342347858,2.429'
b'2.51504373119231,1.0731637904355105,0.5574401201546321,-0.17273513019187772,-0.612912610473286,-0.01909156503651574,-0.5710993036045546,-0.027490309606616956,5.00001'
b'0.09734603446040174,0.7527628439249472,-0.20218964416999152,-0.1954700015215477,-0.4060513603629498,0.006785531677655949,-0.813715166526018,0.656614793197258,1.119'
b'0.04326300977263167,-1.0895425985107923,-0.38878716774583305,-0.10789864528874438,-0.6818663605100649,-0.0723871014747467,-0.8883662012710817,0.8213992340186296,1.426'
b'0.4853051504718848,-0.8492418886278699,-0.06530126513877861,-0.023379656040017353,1.4974350551260218,-0.07790657783453239,-0.9023632702857819,0.7814514907892068,2.956'
b'-1.0635474225567902,1.874166156711919,-0.49344892844525906,-0.06962612737313081,-0.273587577397559,-0.13419514417565354,1.0338979434143465,-1.3457658361775

In [11]:
# tf.io.decode_csv(str, record_defaults)

sample_str= '1,2,3,4,5'
record_defaults = [tf.constant(0, dtype=tf.int32)] *5
parsed_fields = tf.io.decode_csv(sample_str, record_defaults)
print(parsed_fields)

[<tf.Tensor: shape=(), dtype=int32, numpy=1>, <tf.Tensor: shape=(), dtype=int32, numpy=2>, <tf.Tensor: shape=(), dtype=int32, numpy=3>, <tf.Tensor: shape=(), dtype=int32, numpy=4>, <tf.Tensor: shape=(), dtype=int32, numpy=5>]


In [23]:
def parse_csv_line(line, n_fields = 9):
    defs = [tf.constant(np.nan)] * n_fields
    parsed_fields = tf.io.decode_csv(line, record_defaults=defs)
    x = tf.stack(parsed_fields[0:-1])
    y = tf.stack(parsed_fields[-1:])
    return x,y

parse_csv_line(b'0.7751155655229017,1.874166156711919,0.15645971958808144,-0.18905190538070707,-0.6292437617977863,-0.08791603438866835,-0.7483955111240856,0.5717258388347319,4.851',n_fields=9)

(<tf.Tensor: shape=(8,), dtype=float32, numpy=
 array([ 0.77511555,  1.8741661 ,  0.15645972, -0.18905191, -0.6292438 ,
        -0.08791603, -0.7483955 ,  0.57172585], dtype=float32)>,
 <tf.Tensor: shape=(1,), dtype=float32, numpy=array([4.851], dtype=float32)>)

In [24]:
def csv_reader_dataset(filenames, n_readers=5,
                       batch_size=32, n_parse_threads=5,
                       shuffle_buffer_size=10000):
    
        dataset = tf.data.Dataset.list_files(filenames)
        dataset = dataset.repeat()
        dataset = dataset.interleave(
            lambda filename: tf.data.TextLineDataset(filename).skip(1),
            cycle_length = n_readers

        )

        dataset.shuffle(shuffle_buffer_size)
        dataset = dataset.map(parse_csv_line,num_parallel_calls = n_parse_threads)

        dataset = dataset.batch(batch_size)

        return dataset


In [25]:
train_set = csv_reader_dataset(train_filenames, batch_size=3)

In [27]:
for x_batch, y_batch in train_set.take(2):
    print("x:")
    pprint.pprint(x_batch)
    
    print("y:")
    pprint.pprint(y_batch)

x:
<tf.Tensor: shape=(3, 8), dtype=float32, numpy=
array([[ 4.0127665e-01, -9.2934215e-01, -5.3330503e-02, -1.8659453e-01,
         6.5456617e-01,  2.6434466e-02,  9.3125278e-01, -1.4406418e+00],
       [ 4.9710345e-02, -8.4924191e-01, -6.2146995e-02,  1.7878747e-01,
        -8.0253541e-01,  5.0660671e-04,  6.4664572e-01, -1.1060793e+00],
       [-6.6722274e-01, -4.8239522e-02,  3.4529406e-01,  5.3826684e-01,
         1.8521839e+00, -6.1125383e-02, -8.4170932e-01,  1.5204847e+00]],
      dtype=float32)>
y:
<tf.Tensor: shape=(3, 1), dtype=float32, numpy=
array([[2.512],
       [2.286],
       [1.59 ]], dtype=float32)>
x:
<tf.Tensor: shape=(3, 8), dtype=float32, numpy=
array([[-1.0591781 ,  1.3935647 , -0.02633197, -0.1100676 , -0.6138199 ,
        -0.09695935,  0.3247131 , -0.03747724],
       [ 0.48530516, -0.8492419 , -0.06530126, -0.02337966,  1.4974351 ,
        -0.07790658, -0.90236324,  0.78145146],
       [-0.8757754 ,  1.8741661 , -0.94874996, -0.09657185, -0.7163432 ,
        -

In [28]:
batch_size = 32
train_set = csv_reader_dataset(train_filenames,
                               batch_size = batch_size)

valid_set = csv_reader_dataset(valid_filenames,
                               batch_size = batch_size)

test_set = csv_reader_dataset(test_filenames,
                               batch_size = batch_size)

In [29]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation='relu',
                       input_shape=[8]),
    
    keras.layers.Dense(1),
    
])


model.compile(loss="mse", optimizer="sgd")

callbacks = [keras.callbacks.EarlyStopping(
    patience=5, min_delta=1e-2)]

history = model.fit(train_set,
                    validation_data = valid_set,
                    steps_per_epoch = 11600 // batch_size, # 遍歷一次需要多少次迭代
                    validation_steps = 3870 // batch_size,
                    epochs =100,
                    callbacks = callbacks)

Train for 362 steps, validate for 120 steps
Epoch 1/100
362/362 [==============================] - 1s 2ms/step - loss: 1.2680 - val_loss: 0.9409
Epoch 2/100
362/362 [==============================] - 0s 1ms/step - loss: 1.9410 - val_loss: 4.4613
Epoch 3/100
362/362 [==============================] - 0s 1ms/step - loss: 0.4537 - val_loss: 1.8533
Epoch 4/100
362/362 [==============================] - 0s 1ms/step - loss: 0.3963 - val_loss: 1.5497
Epoch 5/100
362/362 [==============================] - 0s 1ms/step - loss: 0.3830 - val_loss: 1.4573
Epoch 6/100
362/362 [==============================] - 0s 1ms/step - loss: 0.3814 - val_loss: 1.3819


In [31]:
model.evaluate(test_set, steps = 5160// batch_size)

161/161 [==============================] - 0s 896us/step - loss: 0.4630


0.4629919808936415

In [37]:
# tfrecord 文件格式
# -> tf.train_Example
#    -> tf.train.Features -> {"key": tf.train.Feature}
#      -> tf.train.Feature -> tf.train.ByteList/FloatList/Int64List

favorite_books = [name.encode('utf-8')
                  for name in ["machine learning", "cc150"]]

favorite_books_bytelist = tf.train.BytesList(value = favorite_books)
print(favorite_books_bytelist)

hours_floatlist = tf.train.FloatList(value= [15.5, 9.5, 7.0 , 8.0])
print(hours_floatlist)

age_int64list = tf.train.Int64List(value= [42])
print(age)

features = tf.train.Features(
    feature = { 
        "favorite_books": tf.train.Feature(
            bytes_list = favorite_books_bytelist),
        "hours":tf.train.Feature(
            float_list = hours_floatlist),
        "age":tf.train.Feature(int64_list= age_int64list),
    }

)


value: "machine learning"
value: "cc150"

value: 15.5
value: 9.5
value: 7.0
value: 8.0

value: 42



In [38]:
print(features)

feature {
  key: "age"
  value {
    int64_list {
      value: 42
    }
  }
}
feature {
  key: "favorite_books"
  value {
    bytes_list {
      value: "machine learning"
      value: "cc150"
    }
  }
}
feature {
  key: "hours"
  value {
    float_list {
      value: 15.5
      value: 9.5
      value: 7.0
      value: 8.0
    }
  }
}



In [39]:
example = tf.train.Example(features = features)
print(example)

serialized_example = example.SerializeToString()
print(serialized_example)

features {
  feature {
    key: "age"
    value {
      int64_list {
        value: 42
      }
    }
  }
  feature {
    key: "favorite_books"
    value {
      bytes_list {
        value: "machine learning"
        value: "cc150"
      }
    }
  }
  feature {
    key: "hours"
    value {
      float_list {
        value: 15.5
        value: 9.5
        value: 7.0
        value: 8.0
      }
    }
  }
}

b'\n\\\n\x1d\n\x05hours\x12\x14\x12\x12\n\x10\x00\x00xA\x00\x00\x18A\x00\x00\xe0@\x00\x00\x00A\n-\n\x0efavorite_books\x12\x1b\n\x19\n\x10machine learning\n\x05cc150\n\x0c\n\x03age\x12\x05\x1a\x03\n\x01*'


In [44]:
output_dir = "tftrcord_basic"

if not os.path.exists(output_dir):
    os.mkdir(output_dir)

filename = "test.tfrecords"

filename_fullpath = os.path.join(output_dir , filename)

with tf.io.TFRecordWriter(filename_fullpath) as writer:
    for i in range(3):
        writer.write(serialized_example)
    

In [45]:
dataset = tf.data.TFRecordDataset([filename_fullpath])

for serialized_example_tensor in dataset:
    print(serialized_example_tensor)

tf.Tensor(b'\n\\\n\x1d\n\x05hours\x12\x14\x12\x12\n\x10\x00\x00xA\x00\x00\x18A\x00\x00\xe0@\x00\x00\x00A\n-\n\x0efavorite_books\x12\x1b\n\x19\n\x10machine learning\n\x05cc150\n\x0c\n\x03age\x12\x05\x1a\x03\n\x01*', shape=(), dtype=string)
tf.Tensor(b'\n\\\n\x1d\n\x05hours\x12\x14\x12\x12\n\x10\x00\x00xA\x00\x00\x18A\x00\x00\xe0@\x00\x00\x00A\n-\n\x0efavorite_books\x12\x1b\n\x19\n\x10machine learning\n\x05cc150\n\x0c\n\x03age\x12\x05\x1a\x03\n\x01*', shape=(), dtype=string)
tf.Tensor(b'\n\\\n\x1d\n\x05hours\x12\x14\x12\x12\n\x10\x00\x00xA\x00\x00\x18A\x00\x00\xe0@\x00\x00\x00A\n-\n\x0efavorite_books\x12\x1b\n\x19\n\x10machine learning\n\x05cc150\n\x0c\n\x03age\x12\x05\x1a\x03\n\x01*', shape=(), dtype=string)


In [ ]:
expected_features = {
    "favorite_books": tf.io.VarLenFeature(dtype = tf.string),
    "hours":tf.io.VarLenFeature(dytpe = tf.float32),
    "age":tf.io.FixedLenFeature([], dtype=tf.int64),
}